In [2]:
# Umwandler mit Fenster zur Dateiabfrage ==>> funktioniert, 
# aber die Umwandlung in die EXE dauert sehr lange.... also bitte GEDULD !!

# Hiermit wird von Notebook in Python und dann in EXE, 
# oder von Python in EXE umgewandelt. 

# Vor Beginn müssen 2 Module installiert werden !
# 1. pip install pyinstaller
# 2. pip install auto-py-to-exe

# Danach kann der Code problemlos ( bei mir zumindest ) ausgeführt werden.


# Importieren der notwenigen Module
import nbformat
from nbconvert import PythonExporter
import subprocess
import chardet
import os
import tkinter as tk
from tkinter import filedialog
import multiprocessing
import threading
import time

# ANSI Escape Codes für Farben in der Konsole
RED = "\033[31m"      # Rot für Fehlermeldungen oder Warnungen
GREEN = "\033[32m"    # Grün für erfolgreiche Meldungen
YELLOW = "\033[33m"   # Gelb für Hinweise oder neutrale Meldungen
BLUE = "\033[34m"     # Blau für Informationsmeldungen
RESET = "\033[0m"     # Setzt die Farbe auf Standard zurück

# Hier wird versucht zu erkennen, mit welchem Encoding der Code generiert wurde
def detect_encoding(file_path):
    try:
        rawdata = open(file_path, "rb").read()
        result = chardet.detect(rawdata)
        charenc = result['encoding']
        return charenc
    except Exception as e:
        print(f"{RED}Fehler bei der Encoding-Erkennung: {str(e)}{RESET}")
        return "utf-8"

# Hier wird die Jupyter-Notebook-Datei *.ipynb in eine Python-Datei *.py umgewandelt
def convert_notebook_to_script(input_file):
    try:
        # Abfrage des Ausgabepfads für die Ziel-Datei als Python-Skript im py Format mit Dateiauswahlfenster
        script_file = filedialog.asksaveasfilename(title="Ausgabe Ziel-Datei als Python-Skript im py Format auswählen", defaultextension=".py", initialfile="Ziel-Datei als Python-Skript im py Format")

        # Überprüfen, ob der Ausgabepfad gültig ist
        if os.path.dirname(script_file) != "" and not os.path.isdir(os.path.dirname(script_file)):
            print(f"{RED}Der angegebene Verzeichnispfad für das Python-Skript existiert nicht.{RESET}")
            create_dir = filedialog.askyesno("Verzeichnis erstellen", "Möchten Sie das Verzeichnis erstellen?")
            if create_dir:
                os.makedirs(os.path.dirname(script_file))
                print(f"{GREEN}Verzeichnis '{os.path.dirname(script_file)}' erstellt.{RESET}")
            else:
                print(f"{BLUE}Bitte geben Sie einen gültigen Pfad ein oder drücken Sie Enter für den Standardpfad.{RESET}")
                script_file = filedialog.asksaveasfilename(title="Ausgabe Datei auswählen", defaultextension=".py", initialfile="Ziel-Datei als Python-Skript im py Format", filetypes=("Jupyter Notebook Datei", "*.ipynb"))

        print(f"\n{BLUE}Schritt 1: Encoding erkennen...{RESET}")
        detected_encoding = detect_encoding(input_file)
        print(f"\n{YELLOW}Erkanntes Encoding : {detected_encoding}{RESET}")

        # Lese das Notebook mit UTF-8 Kodierung
        with open(input_file, 'r', encoding=detected_encoding) as f:
            notebook = nbformat.read(f, as_version=4)

        # Konvertiere das Notebook in ein Python-Skript
        exporter = PythonExporter()
        script, _ = exporter.from_notebook_node(notebook)

        # Speichere das Skript mit UTF-8 Kodierung
        with open(script_file, 'w', encoding='utf-8') as f:
            f.write(script)
        return script_file
    
    except Exception as e:
        print(f"{RED}Fehler beim Konvertieren des Notebooks: {str(e)}{RESET}")
        return None
    
# Hier wird die Python-Datei *.py in eine EXE-Datei *.exe umgewandelt
def convert_to_exe(script_file):
    def run_conversion():
        command = ['pyinstaller', '--onefile', script_file]
        result = subprocess.run(command, capture_output=True, text=True)
        # Das Standartverzeichnis aus pyinstaller ist in der Grundeinstellung immer der Ordner 'dist' im Standard Arbeitspfad
        if result.returncode == 0:
            print(f"\n{GREEN}Erfolgreich in EXE umgewandelt. Die Datei befindet sich im aktuellen Arbeitsverzeichnis in dem 'dist'-Ordner.{RESET}")
        else:
            print(f"{RED}Fehler beim Umwandeln: {result.stderr}{RESET}")
    
    thread = threading.Thread(target=run_conversion)
    thread.start()
    return thread

# Hauptmenue / Startmenue
def main():
    # Abfrage der Jupyter Notebook Datei *.ipynb oder Python Datei *.py mit Dateiauswahlfenster
    input_file = filedialog.askopenfilename(title="Start-Datei Jupyter Notebook Datei *.ipynb oder Python Datei *.py auswählen", filetypes=[("Jupyter Notebook Datei", "*.ipynb"),("Python Datei", "*.py")])

    # Überprüfen, ob die Datei existiert
    while not os.path.isfile(input_file):
        print(f"{RED}Die Datei wurde nicht gefunden. Bitte überprüfen Sie den Pfad.{RESET}")
        input_file = filedialog.askopenfilename(title="Start-Datei Jupyter Notebook Datei *.ipynb oder Python Datei *.py auswählen", filetypes=[("Jupyter Notebook Datei", "*.ipynb"),("Python Datei", "*.py")])

    # überprüft die Datei-Endung auf Notebook .ipynb-Datei oder Python-Skript .py-Datei für die weiteren Schritte
    if input_file.endswith('.ipynb'):
        # hier wird das weitere Vorgehen für die Notebook .ipynb-Datei eingeleitet
        print(f"\n{BLUE}Schritt 2: Notebook .ipynb-Datei in Python-Skript .py-Datei konvertieren...{RESET}")
        script_file = convert_notebook_to_script(input_file)
        if script_file:
            print(f"\n{BLUE}Schritt 3: Python-Skript .py-Datei in EXE-Datei umwandeln...\n{RESET}{RED} !!! BITTE GEDULD HABEN !!!{RESET}")
            conversion_thread = convert_to_exe(script_file)
    elif input_file.endswith('.py'):
        # hier wird das weitere Vorgehen für die Python-Skript .py-Datei eingeleitet
        script_file = input_file
        print(f"\n{BLUE}Python-Skript in EXE-Datei umwandeln...\n{RESET}{RED} !!! BITTE GEDULD HABEN !!!{RESET}")
        conversion_thread = convert_to_exe(script_file)
    else:
        print(f"{RED}Ungültiges Dateiformat. Bitte wählen Sie eine .ipynb-Datei oder .py-Datei.{RESET}")
        return

    # Warten, bis die Umwandlung abgeschlossen ist
    conversion_thread.join()    # wenn zwischen den Klammern ein Wert eingegeben wird, wird diese Anzahl in Sekunden gewartet, bis eine Timeout-Fehlermeldung ausgegeben wird 
    
    print(f"\n{GREEN}Die Umwandlung in eine EXE-Datei war erfolgreich und der Vorgang wurde beendet.{RESET}")

    # Fügt eine Wartezeit hinzu, falls nötig
    print(f"\n{YELlOW}Das Tool wird in 10 Sekunden geschlossen.{RESET}")   
    # Wartet 10 Sekunden nach dem Ende der Konvertierung, damit die ggf. Ausgaben gelesen werden können
    time.sleep(10) 

if __name__ == "__main__":
    # Stellt sicher, dass die Multiprocessing-Funktionalität auf Windows korrekt funktioniert
    multiprocessing.freeze_support()

    # Startet das Hauptprogramm
    main()


ModuleNotFoundError: No module named 'chardet'